In [23]:
from datasets import Dataset,load_dataset

dataset = load_dataset("json",data_files="/data0/yss/sft_data_merge_v3_filter.jsonl",split="train")
dataset2 = load_dataset("json",data_files="/data0/yss/sft_data_merge_v3_filter.jsonl",split="train",streaming=True)

In [24]:
dataset = dataset.train_test_split(test_size=0.1)
dataset["test"].to_json("small_data.jsonl")


Creating json from Arrow format: 100%|██████████| 177/177 [00:08<00:00, 19.83ba/s]


448437831

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/data0/pretrained-models/Qwen2-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)

prompt = "Give me a short introduction to large language model."
messages = [
    # {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
text


/home/qyhuang/miniconda3/envs/dschat/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nGive me a short introduction to large language model.<|im_end|>\n<|im_start|>assistant\n'

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
print(model)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512)
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)